# BIOS512 Assignment #4

Import the 📈Tidyverse into your `R` session

In [1]:
library('tidyverse')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In this assignment we'll be using data from the [TidyTuesday Project](https://github.com/rfordatascience/tidytuesday). Specifically, we'll investigate vaccination rates at US schools. A `CSV` file of the data is located at:  

[https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-02-25/measles.csv](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-02-25/measles.csv)

Get the "raw" data URL and import it into your `R` session using `read_csv`. Remember to capture the data under a variable name of your choosing!

In [2]:
measles = read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-25/measles.csv')
measles |> head()

Rows: 66113 Columns: 16

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): state, year, name, type, city, county
dbl (8): index, enroll, mmr, overall, xmed, xper, lat, lng
lgl (2): district, xrel


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



index state   year    name                                     type   
1 1     Arizona 2018-19 A J Mitchell Elementary                  Public 
2 2     Arizona 2018-19 Academy Del Sol                          Charter
3 3     Arizona 2018-19 Academy Del Sol - Hope                   Charter
4 4     Arizona 2018-19 Academy Of Mathematics And Science South Charter
5 5     Arizona 2018-19 Acclaim Academy                          Charter
6 6     Arizona 2018-19 Alfred F Garcia                          Public 
  city    county     district enroll mmr overall xrel xmed xper lat     
1 Nogales Santa Cruz NA       51     100 -1      NA     NA   NA 31.34782
2 Tucson  Pima       NA       22     100 -1      NA     NA   NA 32.22192
3 Tucson  Pima       NA       85     100 -1      NA     NA   NA 32.13049
4 Phoenix Maricopa   NA       60     100 -1      NA     NA   NA 33.48545
5 Phoenix Maricopa   NA       43     100 -1      NA   2.33 2.33 33.49562
6 Phoenix Maricopa   NA       36     100 -1      NA     NA   NA 33.43532
  lng      
1 -110.9380
2 -110.8961
3 -111.1170
4 -112.1306
5 -112.2247
6 -112.1168

The `mmr` column has holds 🤒 measles, mumps, rubella 🤒 vaccination rates for students in each respective school. **If the `mmr` value is not available for a school the `mmr` value is set to -1 in this data set.** 

The target `mmr` vaccination rate as [recommended by the CDC](https://www.cdc.gov/mmwr/preview/mmwrhtml/mm5632a3.htm) for schools is 95%. 

**Calculate the fraction of schools per state that have vaccination rates greater than the CDC 95%. Capture the output as a table called `df_vacc_rates`.**

You'll need to do use `filter`, `group_by`, and `summarize`. 

I.e.
1. Filter out schools that don't report `mmr` vaccination rate (keep schools where `mmr >= 0`).
1. Group the data by `state`.
1. Summarize fraction of vaccination rates above 95%.

💡Remember `n()` stands for the number or records in a group. Also, `sum(mmr >= 95, na.rm=TRUE)` will count the number of values above or equal to 95.💡

In [16]:
df_vacc_rates = measles |>
    group_by(state) |>
    filter((mmr >= 0)) |>
    summarize(goal = sum(mmr >=95, na.rm = TRUE), frac = sum(mmr >=95, na.rm = TRUE)/n())
   
  
df_vacc_rates |> arrange(frac)

  

state         goal  frac       
1  Arkansas          2 0.003527337
2  Washington      646 0.326592518
3  North Dakota    164 0.455555556
4  Maine           169 0.473389356
5  Minnesota       755 0.473948525
6  Ohio           1435 0.491606715
7  Arizona         593 0.506404782
8  Oregon          478 0.593052109
9  Colorado        939 0.623092236
10 Texas           508 0.626387176
11 Utah            410 0.679933665
12 Missouri        476 0.680000000
13 Vermont         231 0.683431953
14 Montana         370 0.708812261
15 South Dakota    277 0.742627346
16 Connecticut     478 0.811544992
17 Pennsylvania   1619 0.834966478
18 New York       3576 0.859822073
19 Massachusetts   826 0.865828092
20 California    12639 0.888506151
21 Illinois       6893 0.896825397

Which state (of those that report `mmr`) has the smallest fraction of schools above the CDC target vaccination rate of 95%?  

Arkansas

**Make a ECDF plot of the school vaccination rates in North Carolina.** Use the `overall` column which reports the "overall" vaccination rate at each school.

❗️Remember, you can calculate the `y` value for ECDF charts using `row_number`, `mutate`, and `arrange`.

In [26]:
NCvac = measles |>
    filter((state == 'North Carolina'),(overall >= 0)) |>
    arrange(overall) |>
    mutate(ypos = row_number()/n()) |>
    select('overall', 'ypos')

NCvac |> head()


overall ypos        
1 21.7    0.0004798464
2 40.3    0.0009596929
3 42.9    0.0014395393
4 43.5    0.0019193858
5 45.0    0.0023992322
6 45.3    0.0028790787

In [41]:
p = ggplot(NCvac, aes(x = overall, y = ypos)) +
    geom_point(color = 'blue') + geom_line(color = 'blue') +
    xlab(label = 'Vaccination Rate (%)') + ylab(label = 'Fraction of Schools') +
    scale_x_continuous(labels = scales::percent_format(scale = 1)) + 
    geom_vline(xintercept = 95, linetype = "dashed", ) + 
    annotate("text", x = 60, y = 0.75, label = "CDC recommended vaccination rate (95%)")
p

(My solution is below if you want to peek 👀. Uncomment the code, change the cell to `markdown`, and exectute it.)

![](https://github.com/chuckpr/BIOS512/blob/main/assignments/measles-chart.png?raw=true)

Is the median vaccination rate for NC schools above the CDC recommended target?

Yes